## Streaming

### Streaming refers to the process of delivering responses incrementally, as they are generated by the AI model, rather than waiting for the complete response before sending it. Streaming lets you subscribe to updates of the agent run as it proceeds. This can be useful for showing the end-user progress updates and partial responses.

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables")

In [4]:
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent

In [7]:
agent = Agent(
    name = "Joke Agent",
    instructions= "You are a helpful assistant.",
    model= "gpt-4o-mini"
)

result = Runner.run_streamed(agent, "Tell me 5 jokes")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush= True)

Sure! Here are five jokes for you:

1. **Why did the scarecrow win an award?**  
   Because he was outstanding in his field!

2. **What do you call fake spaghetti?**  
   An impasta!

3. **Why did the bicycle fall over?**  
   Because it was two-tired!

4. **What did one wall say to the other wall?**  
   I'll meet you at the corner!

5. **Why don’t scientists trust atoms?**  
   Because they make up everything!  

Hope these made you smile!

In [12]:
from agents import function_tool, ItemHelpers
import random

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)

agent = Agent(
    name = "Joker",
    instructions= "First call the `how_many_jokes` tool, then tell that many jokes.",
    tools=[how_many_jokes]
)

result = Runner.run_streamed(agent, "Hello")

print("=== Run starting ===")

async for event in result.stream_events():
    # We'll ignore the raw responses event deltas
    if event.type == "raw_response_event":
        continue
    # When the agent updates, print that
    elif event.type == "agent_updated_stream_event":
        print(f"Agent updated: {event.new_agent.name}")
        continue
    # When items are generated, print them
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print("-- Tool was called")
        elif event.item.type == "tool_call_output_item":
            print(f"-- Tool output: {event.item.output}")
        elif event.item.type == "message_output_item":
            print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
        else:
            pass  # Ignore other event types

print("=== Run complete ===")

=== Run starting ===
Agent updated: Joker
-- Tool was called
-- Tool output: 6
-- Message output:
 Great! Here are six jokes for you:

1. **Why don't scientists trust atoms?**  
   Because they make up everything!

2. **What do you get when you cross a snowman and a vampire?**  
   Frostbite.

3. **Why was the math book sad?**  
   Because it had too many problems.

4. **What do you call fake spaghetti?**  
   An impasta!

5. **Why did the scarecrow win an award?**  
   Because he was outstanding in his field.

6. **How does a penguin build its house?**  
   Igloos it together! 

Enjoy!
=== Run complete ===
